# Kapitel 4 - Naive Bayes mit Multiclass Klassifizierung

## 4.1. Kapitelübersicht <a class="anchor" id="4-1"/>

In diesem Kapitel betrachten wird die <b>Multiclass Klassifizierung</b>, mit der wir im Gegensatz zu <b>binären Klassifizierung</b> Texte mit mehr als zwei Klassen klassifizieren können. Neben dem bereits bekannten <b>CountVectorizer</b> schauen wir uns noch eine Variation des Bag-of-Words Modells an, den <b>TfidfVectorizer</b>, der Wörter in eine <b>Tf-idf-Gewichtung</b> transformiert. Beide Vectorizer wenden wir sowohl auf den Filmrezensionen Datensatz als auch auf das Wikipedia-Korpus an. Beide Datensätze hatten wir bereits in Kapitel 2 näher betrachtet.

<b>Abschnittsübersicht</b><br>

[4.1. Kapitelübersicht](#4-1)<br>
[4.2. Multiclass Klassifizierung mit dem Bag-of-Words Modell](#4-2)<br>
[4.3. Der Tfidf-Vectorizer](#4-3)<br>
[4.3.1 Anwendung auf die Film-Rezensionen](#4-3-1)<br>
[4.3.2 Anwendung auf das Wikipedia-Korpus](#4-3-2)<br>

Am Ende dieses Kapitel werden wir folgende Themen behandelt und/oder vertieft haben:
- Multiclass Klassifizierung
- Tfidf Vectorizer
- Vergleich von Vectorizern
- Anwendung des Multinomial Naive Bayes auf das Wikipedia-Korpus

## 4.2. Multiclass Klassifizierung mit dem Bag-of-Words Modell <a class="anchor" id="4-2"/>

Bis jetzt hatten wir uns nur eine <b>binäre Klassifizierung</b> (englisch: binary classification) angeguckt. Dies bedeutet, dass die Anzahl der Klassen <i>binär</i> ist: Ein Text gehört entweder zu einer Klasse oder zu einer anderen Klassen. In unserem Fall waren die binären Klassen "Sport" und "Kein Sport". Diese Art der Klassifizierung wird z.b. für die Erkennung von Spam Mails verwendet, bei der Mails in die Kategorien "Spam" und "Kein Spam" unterteilt werden. In der Praxis stoßen wir jedoch öfters auf die Problematik, dass wir 3 oder mehr Klassen vorliegen haben und diese klassifizieren möchten. Dieses Klassifizierungsproblem nennt sich <b>multiclass classification</b> (deutsch: Mehrklassen-Klassifizierung) oder auch <b>multinomial classification</b>.

Bevor wir für die Multiclass Klassifikation das Wikipedia-Korpus benutzen, schauen wir uns das Einstiegsbeispiel aus Kapitel 2 noch einmal an. Hier haben wir 7 Rezensionen zu Filmen, die als Kategorie eine Schulnote von 1-6 erhalten haben.

In [35]:
import pandas as pd

d = {"r1": [1, "Der Film war so genial!"], 
     "r2": [6, "Schlechtester Film aller Zeiten!!!!1!!!1!!"],
     "r3": [3, "Er war schon okay"],
     "r4": [1, "Ich will den Film für den Rest meines Lebens jeden Tag gucken :O"],
     "r5": [2, "Richtig guter Film mit kleineren Schwächen"],
     "r6": [4, "Naja, gibt besseres, aber auch schlechteres, wenn auch nicht viel..."],
     "r7": [5, "Bis auf die Songauswahl war der Film zum Kotzen"]}
corpus = pd.DataFrame.from_dict(d, orient="index", columns=["Note", "Rezension"])
corpus

,Note,Rezension
r1,1,Der Film war so genial!
r2,6,Schlechtester Film aller Zeiten!!!!1!!!1!!
r3,3,Er war schon okay
r4,1,Ich will den Film für den Rest meines Lebens j...
r5,2,Richtig guter Film mit kleineren Schwächen
r6,4,"Naja, gibt besseres, aber auch schlechteres, w..."
r7,5,Bis auf die Songauswahl war der Film zum Kotzen


Wir verwenden unser Klassifizierungsverfahren aus dem vorherigen Kapitel und verändern nur die Spaltenbezeichnungen (`corpus["Kategorie"] → corpus["Note"]; corpus["Text"] → corpus["Rezension"]`). Zudem schauen wir uns hier nicht die kodierte Repräsentation des Voraussage des Klassenlabels an, da dies verwirrend ist, wenn die Klassenlabels Zahlen sind (Die Note "1" wird hier zur 0, die Note "2" wird zur "1", die Note "3" wird zur "2" usw.).

In [36]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

# Kategorien in numerische Repräsentation transformieren
encoder = LabelEncoder()
labels = encoder.fit_transform(corpus["Note"])

# Text in numerische Repräsentation transformieren
vectorizer = CountVectorizer()
vector = vectorizer.fit_transform(corpus["Rezension"])

# Multinomial Naive Bayes
classifier = MultinomialNB()
mnb = classifier.fit(vector, labels)

# Unser Testsatz
vector2 = vectorizer.transform(["Ein sehr knappes Spiel"])

# Die Voraussage
prediction = mnb.predict(vector2)
#print(prediction)
original_predicted_category = encoder.inverse_transform(prediction)
print(str(original_predicted_category))

[1]


Der Satz "Ein sehr knappes Spiel" wird der Note "1" zugeordnet. Diese Zuordnung scheint sehr willkürlich, was daran liegt, dass der Satz im Kontext einer Filmrezension keine wirkliche Bedeutung hat. Versuchen wir es einmal mit Sätzen, die eher einer Filmrezension ähneln.

In [37]:
# Unser Testsatz
review3 = "Ein guter Film"
review4 = "Ein schlechter Film"
review5 = "Der Film war okay"
vector3 = vectorizer.transform([review3])
vector4 = vectorizer.transform([review4])
vector5 = vectorizer.transform([review5])

# Die Voraussage
prediction3 = mnb.predict(vector3)
prediction4 = mnb.predict(vector4)
prediction5 = mnb.predict(vector5)


original_predicted_category3 = encoder.inverse_transform(prediction3)
print(f"Die Rezension '{review3}' steht für die Note " 
      + str(original_predicted_category3) + ".")

original_predicted_category4 = encoder.inverse_transform(prediction4)
print(f"Die Rezension '{review4}' steht für die Note " 
      + str(original_predicted_category4) + ".")

original_predicted_category5 = encoder.inverse_transform(prediction5)
print(f"Die Rezension '{review5}' steht für die Note " 
      + str(original_predicted_category5) + ".")

Die Rezension 'Ein guter Film' steht für die Note [2].
Die Rezension 'Ein schlechter Film' steht für die Note [1].
Die Rezension 'Der Film war okay' steht für die Note [1].


Das Ergebnis mag etwas verwundern. Während die Klassifizierung des ersten Satzes "Ein guter Film" noch sinnvoll erscheint, sind die anderen beiden Klassifizierungen weniger verständlich. Doch woran liegt das? Eine Antwort könnte folgendes Beispiel liefern.

In [38]:
review6 = "Ein schlechtester Film"
vector6 = vectorizer.transform([review6])
prediction6 = mnb.predict(vector6)

original_predicted_category6 = encoder.inverse_transform(prediction6)
print(f"Die Rezension '{review6}' steht für die Note " 
      + str(original_predicted_category6) + ".")

Die Rezension 'Ein schlechtester Film' steht für die Note [6].


Der Satz ist grammatikalisch nicht korrekt, klassifiziert aber besser als der semantisch ähnliche Satz "Ein schlechter Film". Das liegt an der Eigenart des Bag-of-Words Modells: Es teilt Sätze in Wörter auf, die Wortbedeutungen werden jedoch nicht berücksichtigt. Die Wörter "schlechtester" und "schlechter" sind für das Bag-of-Words-Modell genauso ähnlich zueinander wie "schlechter" und "gut". Wenn wir für unsere Rezension also nicht genau die Wörter benutzen, die in den Datensätzen verwendet werden, klassifiziert unser Textklassifikationsverfahren nicht richtig.<br>
Ein weiteres Problem ist die <b>Häufigkeit von Wörtern</b> (englisch: word frequency). Umso häufiger ein Wort in einem Text vorkommt, umso höher ist sein Einfluss auf die Berechnung bei der Naive Bayes Klassifikation.
So lässt sich auch die falsche Klassifikation von "Der Film war okay" erklären. Das Wort "okay" taucht zwar in der Rezension r3 "Er war schon okay" mit der Note "3" auf, das Wort "Film" taucht jedoch in zwei Rezensionen mit der Note "1" auf.

Das Bag-of-Words Modell scheint in der Ausführung, in der wir es kennen, limitiert zu sein. Wörter, die öfters vorkommen, haben einen größeren Einfluss auf die Klassifikation als Wörter, die seltener vorkommen. Selten bieten jedoch häufig auftretende Wörter einen großen Informationsgehalt für die Klassifizierung. Diese Wörter werden auch Stoppwörter genannt. Es gibt zwei Möglichkeiten, den Einfluss der Stoppwörter zu verringern: Entweder werden alle Stoppwörter <u>entfernt</u> oder sie werden <u>skaliert</u>. 

<div class="alert alert-info">
<b>Exkurs:</b> Stoppwörter
    
<b>Stoppwörter</b> (englisch: stop words) sind Wörter, die sehr häufig in Texten auftreten, aber einen niedrigen Informationsgehalt besitzen. Übliche Stoppwörter in der deutschen Sprache sind:
- bestimmte Artikel ("der", "die", "das")
- unbestimmte Artikel ("eine", "einer", "ein", "eines")
- Konjunktionen ("und", "oder", "doch", "weil")
- Präpositionen ("in", "an", "von", "bei")
- Negation ("nicht")

Zudem gibt es noch die <b>Stoppzeichen</b>, die oft auch unter die Kategorie der Stoppwörter fallen. Beispiele wären der Punkt ("."), das Komma (",") oder der Strichpunkt (";").

Da wir aktuell nur einen sehr kleinen Datensatz behandeln, würde nach eine Entfernung der Stoppwörter dieser noch kleiner werden. Deshalb behandeln wir nun die Methode der <u>Skalierung</u>. Dafür verwenden wir das <b>Tf-Idf-Maß</b>.

<div class="alert alert-info">
<b>Exkurs:</b> Tf-Idf-Maß
    
Das <b>Tf-Idf-Maß</b> kommt eigentlich aus dem Bereich des <b>Information Retrieval</b> und wird dort zur Beurteilung der Relevanz von Termen in Dokumenten einer Kollektion von Dokumenten eingesetzt.<br>
Das Maß besteht aus zwei Kompontenten:<br>
- <b>tf</b> (= term frequency, deutsch: Termhäufigkeit)
- <b>idf</b> (= inverse document frequency, deutsch: invertierte Dokumenthäufigkeit)

Die Termhäufigkeit <b>tf</b> kann in mehreren Variationen vorliegen. Die einfachste Variante wird <b>"einfache Termhäufigkeit"</b> genannt. Dabei wird wie beim Bag-of-Words Modell die Häufigkeit der Wörter gezählt. Häufigere Wörter in einem Text sind somit wichtiger. Eine weitere Variante wäre die <b>normierte Termhäufigkeit</b>, bei der die Häufigkeit eines Wortes <i>f</i> durch die maximale Worthäufigkeit (also die Häufigkeit des häufigsten Wortes) geteilt wird. Diese Variationen sollen durch zwei Beispiele genauer erläutert werden.<br>

<u>Beispiele</u>:<br>
Wir haben die folgenden Sätze vorliegen:<br>
$ s_1 $= "groß groß stark stark" <br>
$ s_2 $= "groß groß groß groß klein klein klein stark" <br>
Um mit ihnen rechnen zu können, müssen wir sie in Vektoren umwandeln. Dazu müssen wir zuerst einen Vektorraum[<sup>1</sup>](#fn1) bestimmen. Die Dimensionalität dieses Raums wird durch die verschiedenen Wörter unseres Datensätze (hier die Sätze $ s_1 $ und $ s_2 $) bestimmt. Unser Raum ist dreidimensional, da unser Datensatz nur drei verschiedene Wörter enthält: groß, stark, klein. Wir definieren unseren Vektorraum also folgendermaßen: (groß, stark, klein).<br><br>

Bei der <b>einfachen Termhäufigkeit</b> würden die Vektoren für die Sätze $ s_1 $ und $ s_2 $ folgendermaßen aussehen:<br>
$ s_1 = (2, 0, 2) $<br>
$ s_2 = (4, 3, 1) $<br>

Die Häufigkeit der Wörter in den Sätzen wird gezählt und dann in den Vektorraum <b>(groß, stark, klein)</b> "eingesetzt".<br><br>

Bei der <b>normierten Termhäufigkeit</b> würden die Vektoren für die Sätze $ s_1 $ und $ s_2 $ folgendermaßen aussehen:<br>
$ s_1 = (1, 0, 1) $<br>
$ s_2 = (1, \frac{3}{4}, \frac{1}{4}) $<br>

Für jeden Satz wird zuerst das häufigste Wort im Satz bestimmt. Im ersten Satz $ s_1 $ kommen die Wörter "groß" und "stark" jeweils 2 Mal vor. Dies ist der Wert, durch den alle einfachen Termhäufigkeiten geteilt werden. Der Satz $ s_1 $ bei der einfachen Termhäufigkeit wird also zu (1, 0, 1), da $ s_1 = (\frac{2}{2}, \frac{0}{2}, \frac{2}{2}) = (1, 0, 1) $. Genauso funktioniert es beim zweiten Satz. Das häufigste Wort dort ist "groß", es kommt 4 Mal vor. Alle einfachen Termhäufigkeiten werden also durch 4 geteilt und man erhält $ (1, \frac{3}{4}, \frac{1}{4}) $.<br><br>

Die Formel für die invertiere Dokumenthäufigkeit <b>idf</b> ist $ log(\frac{N}{df_i}) $. $ N $ ist die Anzahl aller Texte oder Dokumente. $ {df_i} $ ist die Dokumenthäufigkeit des Worts $ i $, d.h. die Anzahl der Dokumente, die das Wort $ i $ enthalten. Der Logarithmus wird hier genutzt, um die Auswirkung von sehr häufigen Wörtern zu dämpfen. Dazu schauen wir uns die Formel für die <b>tf-idf-Gewichtung</b> an: $ tf \cdot idf $. Diese Gewichtung gibt jedem Wort in unseren Datensätzen ein Gewicht. Worte, die sehr häufig in allen Texten vorkommen, erhalten eine sehr niedrige oder gar keine Gewichtung. Wörter, die sehr häufig in einem bestimmten Dokument vorkommen, aber selten in anderen Texten, erhalten ein sehr hohes Gewicht.<br>

<u>Beispiel</u>:<br>
Wir haben den Satz mit den folgenden Wörtern vorliegen: $ s3 = "groß, stark, klein, stark, stark" $. Für jedes Wort dieses Satzes berechnen wir die einfache Termhäufigkeit:<br>
$ tf_{groß} = 1$<br>
$ tf_{stark} = 3$<br>
$ tf_{klein} = 1$<br>
Als Vektor im Vektorraum (groß, stark, klein) würde dies $ (1, 3, 1) $  geschrieben werden.<br>

Wir nehmen nun an, dass wir 10000 Texte haben. Für jedes Wort haben wir nun die Dokumenthäufigkeit gezählt, also in wievielen Dokumenten das Wort vorkommt. Das Ergebnis ist:<br>
$ df_{groß} = 50$<br>
$ df_{stark} = 1300$<br>
$ df_{klein} = 250$<br>

Nun setzen wir unsere Werte in die Formel für die <b>tf-idf-Gewichtung</b> ($ tf \cdot log(\frac{N}{df_i}) $) ein (wir verwenden hier den natürlichen Logarithmus):<br>
$ tf\text{-}idf_{groß} = 1 \cdot log(\frac{10000}{50}) = 1 \cdot 5.3 = 5.3 $<br>
$ tf\text{-}idf_{stark} = 3 \cdot log(\frac{10000}{1300}) = 3 \cdot 2.0 = 6.0 $<br>
$ tf\text{-}idf_{klein} = 1 \cdot log(\frac{10000}{250}) = 1 \cdot 3.7 = 3.7 $<br>

Angenommen, wir hätten nun ein Wort wie "der", welches die Termhäufigkeit 350 und die Dokumenthäufigkeit 10000 besitzen würde. Da es in allen Dokumenten vorkommt, wäre die tf-idf-Gewichtung folgende:<br>
$ tf\text{-}idf_{der} = 350 \cdot log(\frac{10000}{10000}) = 350 \cdot log(1) = 350 \cdot 0 = 0 $<br>
Das Besonderheit des Logarithmus wird hier ausgenutzt, da $log(1)$ immer null ist. Wörter, die in (beinahe) allen Texten vorkommen wie Stoppwörter werden also gedämpft und erhalten eine niedrige oder gar keine Gewichtung.


<hr style="border: 0.1px solid black;"/>
<span id="fn1" style="font-size:8pt; line-height:1"><sup style="font-size:5pt">1</sup> &nbsp; 
Ich greife hier auf das Vektorraummodell aus dem Bereich des <b>Information Retrieval</b> zurück. Da es hier nur benutzt wird, um das tf-idf-Maß zu erklären, wird es nicht weiter behandelt. Vertiefende Informationen zum Vektorraummodell bietet z.B. die <a href="https://de.wikipedia.org/wiki/Vektorraum-Retrieval">Wikipedia</a>-Seite des Vektorraummodells.</span>

## 4.3. Der Tf-idf Vectorizer <a class="anchor" id="4-3"/>

### 4.3.1. Anwendung auf die Film-Rezensionen <a class="anchor" id="4-3-1"/>

Anstatt des Bag-of-Words Modells verwenden wir nun das Tf-idf-Maß. Dazu tauschen wir den `CountVectorizer` durch den `TfidfVectorizer` aus.

In [39]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

# Kategorien in numerische Repräsentation transformieren
encoder = LabelEncoder()
labels = encoder.fit_transform(corpus["Note"])

# Text in numerische Repräsentation transformieren
vectorizer = TfidfVectorizer()
vector = vectorizer.fit_transform(corpus["Rezension"])

# Multinomial Naive Bayes
classifier = MultinomialNB()
mnb = classifier.fit(vector, labels)

# Unser Testsatz
vector2 = vectorizer.transform(["Ein sehr knappes Spiel"])

# Die Voraussage
prediction = mnb.predict(vector2)
#print(prediction)
original_predicted_category = encoder.inverse_transform(prediction)
print(str(original_predicted_category))

[1]


Alternativ könnten wir mit Scikit learn auch erst den `CountVectorizer` nutzen und diesen dann mit dem `TfIdfTransformer` transformieren. Dies ist m. E. aber umständlicher als den `TfidfVectorizer` direkt zu nutzen und produziert nur mehr Code.

In [40]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

# Kategorien in numerische Repräsentation transformieren
encoder = LabelEncoder()
labels = encoder.fit_transform(corpus["Note"])

# Text in numerische Repräsentation transformieren
vectorizer = CountVectorizer()
vector = vectorizer.fit_transform(corpus["Rezension"])
transformer = TfidfTransformer()
vector = transformer.fit_transform(vector)

# Multinomial Naive Bayes
classifier = MultinomialNB()
mnb = classifier.fit(vector, labels)

# Unser Testsatz
vector2 = vectorizer.transform(["Ein sehr knappes Spiel"])

# Die Voraussage
prediction = mnb.predict(vector2)
#print(prediction)
original_predicted_category = encoder.inverse_transform(prediction)
print(str(original_predicted_category))

[1]


In [41]:
# Unser Testsatz
review3 = "Ein guter Film"
review4 = "Ein schlechter Film"
review5 = "Der Film war okay"
vector3 = vectorizer.transform([review3])
vector4 = vectorizer.transform([review4])
vector5 = vectorizer.transform([review5])

# Die Voraussage
prediction3 = mnb.predict(vector3)
prediction4 = mnb.predict(vector4)
prediction5 = mnb.predict(vector5)


original_predicted_category3 = encoder.inverse_transform(prediction3)
print(f"Die Rezension '{review3}' steht für die Note " 
      + str(original_predicted_category3) + ".")

original_predicted_category4 = encoder.inverse_transform(prediction4)
print(f"Die Rezension '{review4}' steht für die Note " 
      + str(original_predicted_category4) + ".")

original_predicted_category5 = encoder.inverse_transform(prediction5)
print(f"Die Rezension '{review5}' steht für die Note " 
      + str(original_predicted_category5) + ".")

Die Rezension 'Ein guter Film' steht für die Note [1].
Die Rezension 'Ein schlechter Film' steht für die Note [1].
Die Rezension 'Der Film war okay' steht für die Note [1].


Die Ergebnisse sind hier tatsächlich ungenauer als beim `CountVectorizer`. Das könnte daran liegen, dass unser Datensatz sehr klein ist und nur sehr wenige Wörter beinhaltet. Um einen Unterschied zwischen den Vectorizern zu erkennen, brauchen wir einen größeren Datensatz.

### 4.3.2. Anwendung auf das Wikipedia-Korpus <a class="anchor" id="4-3-2"/>

In [1]:
import pandas as pd
corpus = pd.read_csv("tutorialdata/corpora/wikicorpus_v2.csv", index_col=0)
corpus.head()

,id,category,length,text
0,3470,Album nach Typ,1050,All the Best ! ( englisch Alles Gute ! ) ist d...
1,3611,Album nach Typ,525,Let It Roll : Songs by George Harrison ist das...
2,3612,Album nach Typ,251,Lieder wie Orkane ist das dritte offizielle Be...
3,3613,Album nach Typ,756,Long Stick Goes Boom : The Anthology ist eine ...
4,3614,Album nach Typ,260,Los Grandes Éxitos en Español ( spanisch für D...


#### TfidfVectorizer

Wir verwenden unser Klassifizierungsverfahren aus dem vorherigen Abschnitt und verändern nur die Spaltenbezeichnungen (`corpus["Note"] → corpus["category"]; corpus["Rezension"] → corpus["text"]`). Als Beispielsatz nehmen wir erneut den Satz "Ein sehr knappes Spiel".

In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

# Kategorien in numerische Repräsentation transformieren
encoder = LabelEncoder()
labels = encoder.fit_transform(corpus["category"])

# Text in numerische Repräsentation transformieren
vectorizer = TfidfVectorizer()
vector = vectorizer.fit_transform(corpus["text"])

# Multinomial Naive Bayes
classifier = MultinomialNB()
mnb = classifier.fit(vector, labels)

# Unser Testsatz
vector2 = vectorizer.transform(["Ein sehr knappes Spiel"])

# Die Voraussage
prediction = mnb.predict(vector2)
#print(prediction)
original_predicted_category = encoder.inverse_transform(prediction)
print(str(original_predicted_category))

['Computerspiel nach Plattform']


#### CountVectorizer im Vergleich

In [44]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

# Kategorien in numerische Repräsentation transformieren
encoder = LabelEncoder()
labels = encoder.fit_transform(corpus["category"])

# Text in numerische Repräsentation transformieren
vectorizer = CountVectorizer()
vector = vectorizer.fit_transform(corpus["text"])

# Multinomial Naive Bayes
classifier = MultinomialNB()
mnb = classifier.fit(vector, labels)

# Unser Testsatz
vector2 = vectorizer.transform(["Ein sehr knappes Spiel"])

# Die Voraussage
prediction = mnb.predict(vector2)
#print(prediction)
original_predicted_category = encoder.inverse_transform(prediction)
print(str(original_predicted_category))

['Computerspiel nach Plattform']


Der Satz "Ein sehr knappes Spiel" wurde der Kategorie "Computerspiel nach Plattform" bei beiden Vectorizern zugeordnet. Um bewerten zu können, ob diese Einteilung sinnvoll erscheint, schauen wir uns einmal die möglichen Kategorien an.

#### Bewertung der Einteilung

In [45]:
print(corpus.category.unique())

['Album nach Typ' 'Alternative-Rock-Band' 'Burg in Deutschland'
 'Chemikaliengruppe' 'Computerspiel nach Plattform'
 'Deutsche Vereine nach Bundesland' 'Einzelsprache'
 'Fernsehserie nach Staat' 'Film nach Staat' 'Frauenfußball'
 'Gemälde nach Jahrhundert' 'Herrscher nach Titel' 'Internet'
 'Kostümkunde' 'Krankheit' 'Kreditgeschäft' 'Krieg nach Typ'
 'Kunst (Deutschland)' 'Literatur (Romantik)' 'Literaturwissenschaft'
 'Logik' 'Millionenstadt' 'Oper' 'PKW-Modell' 'Planung und Organisation'
 'See in Deutschland' 'Soziologie' 'Wald' 'Western' 'Wirtschaft']


Viele der Kategorien scheinen nicht zum Satz "Ein sehr knappes Spiel" zu passen. In unserem ursprünglichen Beispiel wurde der Satz der Kategorie "Sport" zugewiesen. Eine verwandte Kategorie befindet sich auch in diesem Datensatz: "Frauenfußball". Anstatt der Kategorie "Frauenfußball" wurde der Satz jedoch der Kategorie "Computerspiel nach Plattform" zugeordnet. Um eine Erklärung dafür zu finden, schauen wir uns die Texte der beiden Kategorien genauer an.

#### Kategorie: Frauenfußball

In [46]:
frauenfußball = corpus.loc[corpus["category"] == "Frauenfußball"]
frauenfußball = frauenfußball.drop(["id"], axis=1)
frauenfußball.head()

,category,length,text
1800,Frauenfußball,2236,Frauenfußball ist auch in Deutschland eine der...
1801,Frauenfußball,848,Der Artikel beinhaltet eine ausführliche Darst...
1802,Frauenfußball,355,Die Newcastle United Jets Women sind ein austr...
1803,Frauenfußball,168,Der Verein für Rasensport Niederfell 1949 e.V ...
1804,Frauenfußball,1182,Der Artikel beinhaltet eine ausführliche Darst...


In [47]:
from collections import Counter
from nltk import word_tokenize
frauenfußball_text = frauenfußball.text.to_string()
frauenfußball_counter = dict(Counter(word_tokenize(frauenfußball_text)))

In [48]:
sentence = ["Ein", "sehr", "knappes", "Spiel"]
for word in sentence:
    
    if word in frauenfußball_counter:
        print(f"Das Wort '{word}' kommt " + str(frauenfußball_counter[word]) + " Mal vor.")
    else:
        print(f"Das Wort '{word}' kommt kein einziges Mal vor.")

Das Wort 'Ein' kommt kein einziges Mal vor.
Das Wort 'sehr' kommt kein einziges Mal vor.
Das Wort 'knappes' kommt kein einziges Mal vor.
Das Wort 'Spiel' kommt kein einziges Mal vor.


Keines der Wörter des Satzes "Ein sehr knappes Spiel" kommt in den Texten der Kategorie "Frauenfußball" vor.

#### Kategorie: Computerspiel nach Plattform

In [49]:
computerspiel = corpus.loc[corpus["category"] == "Computerspiel nach Plattform"]
computerspiel = computerspiel.drop(["id"], axis=1)
computerspiel.head()

,category,length,text
800,Computerspiel nach Plattform,353,Ballyhoo ist ein Computerspiel der US-amerikan...
801,Computerspiel nach Plattform,379,Balance of Power ist ein Computer-Strategiespi...
802,Computerspiel nach Plattform,415,Ballblazer ist ein Zweispieler-Computer-Sports...
803,Computerspiel nach Plattform,417,Barbarian : The Ultimate Warrior und Barbarian...
804,Computerspiel nach Plattform,359,Beyond Zork ( kompletter Titel Beyond Zork : T...


In [50]:
from collections import Counter
from nltk import word_tokenize
computerspiel_text = computerspiel.text.to_string()
computerspiel_counter = dict(Counter(word_tokenize(computerspiel_text)))

In [51]:
sentence = ["Ein", "sehr", "knappes", "Spiel"]
for word in sentence:
    
    if word in computerspiel_counter:
        print(f"Das Wort '{word}' kommt " + str(computerspiel_counter[word]) + " Mal vor.")
    else:
        print(f"Das Wort '{word}' kommt kein einziges Mal vor.")

Das Wort 'Ein' kommt kein einziges Mal vor.
Das Wort 'sehr' kommt kein einziges Mal vor.
Das Wort 'knappes' kommt kein einziges Mal vor.
Das Wort 'Spiel' kommt 1 Mal vor.


Bis auf das Wort "Spiel" kommt keines der anderen Wörter in den Texten der Kategorie "Computerspiel nach Plattform" vor. Da jedoch keines der Wörter in der Kategorie "Frauenfußball" vorkam, wurde für den Satz "Ein sehr knappes Spiel" die Kategorie "Computerspiel nach Plattform" gewählt.<br>

Hierbei wurden nun einige Dinge nicht beachtet:
- Art der Datensätze (Nachrichten Artikel, Lexikon Artikel)
- Groß- und Kleinschreibung ("ein", "Ein")
- Kompositionen ("Spiel", "Spielereihe")
- Rechtschreibfehler ("Spiel", "Spieel")
- Angefügte Zeichen ("Spiel", "Spiel-")<br>

Eine genaue Erklärung, warum das Klassifizierungsverfahren eine bestimmte Kategorie für einen Satz oder einen Text genommen wurde, lässt sich nicht immer bestimmen. Vor allem bei größeren Datensätzen ist eine Nachverfolgung müheselig oder kompliziert. Um ein Klassifikationsverfahren und seine Klassifizierung bewerten zu können, brauchen wir <b>Evaluations</b> Methoden.